# Chapter 9: Inheritance & Polymorphism

In the previous chapters, you learned how to create classes and objects, and how to control their initialization. Now we’ll explore one of the most powerful concepts in object‑oriented programming: **inheritance**. Inheritance allows you to define a new class based on an existing class, reusing and extending its functionality. This leads to hierarchical classifications and code reuse.

Closely related to inheritance is **polymorphism** – the ability of an object to take many forms. In C#, polymorphism enables you to write code that works with objects of a base class, but at runtime it can execute methods defined in derived classes.

In this chapter, you’ll learn:

- How to create **base classes** and **derived classes**.
- The role of the `base` keyword.
- How to override methods with `virtual` and `override`.
- The use of **abstract classes** and **abstract methods**.
- How to prevent further inheritance with `sealed`.
- How polymorphism makes your code flexible and extensible.

By the end, you’ll be able to design class hierarchies that model real‑world relationships and write code that adapts to different object types.

---

## 9.1 Base Classes and Derived Classes

Inheritance models an **“is‑a”** relationship. For example, a `Car` is a `Vehicle`, a `Dog` is an `Animal`. The more general class is the **base class** (or parent class), and the more specific class is the **derived class** (or child class).

### Defining a Base Class

```csharp
public class Vehicle
{
    public string Make { get; set; }
    public string Model { get; set; }
    public int Year { get; set; }

    public void Start()
    {
        Console.WriteLine("Vehicle is starting...");
    }
}
```

### Defining a Derived Class

You specify the base class using a colon (`:`) after the derived class name.

```csharp
public class Car : Vehicle
{
    public int NumberOfDoors { get; set; }

    public void Honk()
    {
        Console.WriteLine("Beep beep!");
    }
}
```

Now a `Car` has all the members of `Vehicle` plus its own:

```csharp
Car myCar = new Car();
myCar.Make = "Toyota";      // from Vehicle
myCar.Model = "Corolla";    // from Vehicle
myCar.Year = 2022;          // from Vehicle
myCar.NumberOfDoors = 4;    // from Car
myCar.Start();               // from Vehicle
myCar.Honk();                // from Car
```

### The `base` Keyword

Inside a derived class, you can access members of the base class using the `base` keyword. This is especially useful when you want to call a base class constructor or a method that has been overridden.

```csharp
public class Car : Vehicle
{
    public Car(string make, string model, int year, int doors) 
        : base() // calls base class constructor (implicitly even if omitted)
    {
        // Alternatively, you can call a base constructor with parameters
    }

    public void ShowDetails()
    {
        base.Start(); // explicitly calling base method (optional)
        Console.WriteLine($"Car: {Make} {Model}, {Year} with {NumberOfDoors} doors");
    }
}
```

If the base class has a parameterized constructor, you must call it explicitly using `base(...)`.

```csharp
public class Vehicle
{
    public Vehicle(string make)
    {
        Make = make;
    }
}

public class Car : Vehicle
{
    public Car(string make, string model) : base(make)
    {
        Model = model;
    }
}
```

### Inheritance and Constructors

When you create an instance of a derived class, the base class constructor is called first (either implicitly the default constructor, or the one you specify with `base`). This ensures that the base part of the object is initialized before the derived part.

---

## 9.2 Virtual and Override Methods

By default, methods in a base class are **not** overridable. If you want to allow a derived class to provide its own implementation, you mark the base method as `virtual`. The derived class then uses `override` to replace it.

```csharp
public class Vehicle
{
    public virtual void Start()
    {
        Console.WriteLine("Vehicle is starting...");
    }
}

public class Car : Vehicle
{
    public override void Start()
    {
        Console.WriteLine("Car engine roars to life!");
    }
}
```

Now when you call `Start` on a `Car` object, the overridden version runs.

```csharp
Vehicle myVehicle = new Car(); // a Car stored in a Vehicle variable
myVehicle.Start(); // Output: "Car engine roars to life!"
```

### Override vs. New

If you don't use `override`, but instead use the `new` keyword, you hide the base method. This is rarely needed and can lead to confusion.

```csharp
public class Car : Vehicle
{
    public new void Start() // hides the base method
    {
        Console.WriteLine("Car start (hiding)");
    }
}
```

With hiding, the method called depends on the compile‑time type, not the runtime type. This breaks polymorphism, so use `override` unless you have a specific reason not to.

### Calling Base Implementation

Sometimes you want to extend the base behavior, not replace it entirely. You can call the base method using `base.MethodName()`.

```csharp
public class Car : Vehicle
{
    public override void Start()
    {
        base.Start(); // first do what the base does
        Console.WriteLine("Car specific startup complete.");
    }
}
```

---

## 9.3 Abstract Classes and Methods

Sometimes you want to define a base class that should never be instantiated – it’s too general. Such a class is called an **abstract class**. You mark it with the `abstract` keyword.

An abstract class can have **abstract methods** – methods that have no implementation in the base class but must be implemented by derived classes.

```csharp
public abstract class Shape
{
    public abstract double GetArea(); // no body, just a signature

    public void Display()
    {
        Console.WriteLine($"Area: {GetArea()}");
    }
}

public class Circle : Shape
{
    public double Radius { get; set; }

    public override double GetArea()
    {
        return Math.PI * Radius * Radius;
    }
}
```

**Key points:**

- You cannot create an instance of an abstract class (`new Shape()` is illegal).
- Any non‑abstract derived class must provide implementations for all abstract members.
- Abstract methods are implicitly virtual – they must be overridden.
- An abstract class can have non‑abstract (concrete) members as well.

Abstract classes are perfect for defining a common contract and shared functionality that derived classes can build upon.

---

## 9.4 Sealed Classes and Methods

If you want to prevent further inheritance, mark a class as `sealed`. No class can derive from a sealed class.

```csharp
public sealed class FinalClass
{
    // ...
}

// This causes a compiler error
// public class Derived : FinalClass { }
```

You can also seal an individual `override` method to prevent it from being overridden further down the hierarchy.

```csharp
public class Animal
{
    public virtual void Speak() { }
}

public class Dog : Animal
{
    public sealed override void Speak()
    {
        Console.WriteLine("Woof!");
    }
}

public class Puppy : Dog
{
    // Cannot override Speak because it's sealed in Dog
}
```

Sealing is a design decision: it locks the behavior and can sometimes improve performance (since the runtime knows the method won’t be overridden). Use it when you want to finalize an implementation.

---

## 9.5 Polymorphism in Action

**Polymorphism** means “many forms.” In C#, it allows you to treat objects of derived classes as objects of the base class, and at runtime the correct overridden methods are called.

```csharp
public class Animal
{
    public virtual void MakeSound() => Console.WriteLine("Some sound");
}

public class Dog : Animal
{
    public override void MakeSound() => Console.WriteLine("Woof!");
}

public class Cat : Animal
{
    public override void MakeSound() => Console.WriteLine("Meow!");
}

// Usage
Animal myDog = new Dog();
Animal myCat = new Cat();

myDog.MakeSound(); // Woof!
myCat.MakeSound(); // Meow!
```

Even though the variables are of type `Animal`, the actual object type determines which `MakeSound` runs.

### Polymorphic Collections

You can create a collection of the base type and store any derived objects.

```csharp
List<Animal> animals = new List<Animal>
{
    new Dog(),
    new Cat(),
    new Dog()
};

foreach (Animal a in animals)
{
    a.MakeSound(); // each animal makes its own sound
}
```

This is incredibly powerful: you can write code that works with the base abstraction and automatically handles any new derived types added later.

---

## 9.6 Putting It All Together: A Practical Example

Let’s build an employee hierarchy using inheritance, abstract classes, virtual methods, and polymorphism.

```csharp
using System;
using System.Collections.Generic;

namespace InheritanceDemo
{
    // Abstract base class
    public abstract class Employee
    {
        public string Name { get; set; }
        public int Id { get; set; }

        protected Employee(string name, int id)
        {
            Name = name;
            Id = id;
        }

        // Abstract method – must be implemented by derived classes
        public abstract decimal CalculateMonthlyPay();

        // Virtual method – can be overridden
        public virtual void DisplayInfo()
        {
            Console.WriteLine($"ID: {Id}, Name: {Name}");
        }
    }

    // SalariedEmployee – fixed monthly salary
    public class SalariedEmployee : Employee
    {
        public decimal AnnualSalary { get; set; }

        public SalariedEmployee(string name, int id, decimal annualSalary) 
            : base(name, id)
        {
            AnnualSalary = annualSalary;
        }

        public override decimal CalculateMonthlyPay()
        {
            return AnnualSalary / 12;
        }

        public override void DisplayInfo()
        {
            base.DisplayInfo();
            Console.WriteLine($"Type: Salaried, Annual Salary: {AnnualSalary:C}");
        }
    }

    // HourlyEmployee – pay based on hours worked
    public class HourlyEmployee : Employee
    {
        public decimal HourlyRate { get; set; }
        public double HoursWorked { get; set; }

        public HourlyEmployee(string name, int id, decimal hourlyRate) 
            : base(name, id)
        {
            HourlyRate = hourlyRate;
        }

        public override decimal CalculateMonthlyPay()
        {
            return HourlyRate * (decimal)HoursWorked;
        }

        public override void DisplayInfo()
        {
            base.DisplayInfo();
            Console.WriteLine($"Type: Hourly, Hourly Rate: {HourlyRate:C}, Hours: {HoursWorked}");
        }
    }

    // CommissionEmployee – base salary plus commission on sales
    public class CommissionEmployee : Employee
    {
        public decimal BaseSalary { get; set; }
        public decimal CommissionRate { get; set; }
        public decimal Sales { get; set; }

        public CommissionEmployee(string name, int id, decimal baseSalary, decimal commissionRate) 
            : base(name, id)
        {
            BaseSalary = baseSalary;
            CommissionRate = commissionRate;
        }

        public override decimal CalculateMonthlyPay()
        {
            return BaseSalary + (Sales * CommissionRate);
        }

        public override void DisplayInfo()
        {
            base.DisplayInfo();
            Console.WriteLine($"Type: Commission, Base: {BaseSalary:C}, Rate: {CommissionRate:P}, Sales: {Sales:C}");
        }
    }

    class Program
    {
        static void Main()
        {
            // Create a list of employees (polymorphic collection)
            List<Employee> employees = new List<Employee>
            {
                new SalariedEmployee("Alice", 101, 60000m),
                new HourlyEmployee("Bob", 102, 20m) { HoursWorked = 160 },
                new CommissionEmployee("Charlie", 103, 2000m, 0.05m) { Sales = 50000m }
            };

            // Process each employee polymorphically
            foreach (Employee emp in employees)
            {
                emp.DisplayInfo();
                Console.WriteLine($"Monthly Pay: {emp.CalculateMonthlyPay():C}");
                Console.WriteLine();
            }

            // Demonstrate calling specific member if needed (using type checking)
            Console.WriteLine("--- Overtime for hourly employees ---");
            foreach (Employee emp in employees)
            {
                if (emp is HourlyEmployee hourly)
                {
                    Console.WriteLine($"{hourly.Name} worked {hourly.HoursWorked} hours.");
                }
            }
        }
    }
}
```

**Explanation:**

- `Employee` is an **abstract class** with an abstract method `CalculateMonthlyPay`. It also has a virtual method `DisplayInfo` that can be overridden.
- `SalariedEmployee`, `HourlyEmployee`, and `CommissionEmployee` derive from `Employee` and implement `CalculateMonthlyPay` (and optionally override `DisplayInfo`).
- In `Main`, we store different employee types in a `List<Employee>` – this is polymorphism.
- Iterating over the list, we call `DisplayInfo` and `CalculateMonthlyPay`. Each object behaves according to its actual type.
- We also show how to check the runtime type using `is` and pattern matching to access type‑specific members (like `HoursWorked` for hourly employees).

This example illustrates how inheritance and polymorphism lead to clean, extensible code. Adding a new employee type (e.g., `InternEmployee`) would require no changes to the processing loop – just define the new class and it will work.

---

## 9.7 Common Pitfalls and Best Practices

### 1. Favor Composition Over Inheritance

Not every relationship should be modeled with inheritance. If the relationship is “has‑a” rather than “is‑a”, use composition (embedding an object as a field). Inheritance creates tight coupling; composition is more flexible.

### 2. Use `virtual` Judiciously

Mark methods as `virtual` only when you intend them to be overridden. By default, methods should be non‑virtual to avoid unintended changes in behavior.

### 3. Follow the Liskov Substitution Principle

Derived classes should be substitutable for their base classes without breaking the program. That means:
- Don’t strengthen preconditions (e.g., require more than the base method).
- Don’t weaken postconditions (e.g., return a narrower range).
- Preserve the base class invariants.

Violating this principle leads to brittle code.

### 4. Seal Classes Unless Designed for Inheritance

If you haven’t designed a class to be inherited from, consider marking it `sealed`. This prevents others from deriving from it in ways you didn’t anticipate, which could introduce bugs. The .NET framework does this for many classes.

### 5. Use Abstract Classes for Shared State and Behavior

Abstract classes are useful when you have a common base with some implemented methods and fields that derived classes should share. If you only need a contract, consider using an interface instead (Chapter 10).

### 6. Call Base Methods When Overriding

If you override a method, decide whether you need to call the base implementation. Forgetting to call it might skip essential base logic. Conversely, calling it when not needed can cause unintended side effects.

### 7. Avoid Deep Inheritance Hierarchies

Deep hierarchies are hard to understand and maintain. Aim for shallow, focused hierarchies. If you find yourself needing many levels, reconsider your design.

### 8. Use `sealed` on Overrides When Appropriate

If you want to allow overriding in one derived class but prevent further overriding, seal that override. This locks the behavior at that level.

### 9. Constructor in Inheritance

Remember that base class constructors are called first. If the base class doesn’t have a parameterless constructor, derived classes must explicitly call a base constructor using `base(...)`.

### 10. Prefer Pattern Matching Over `as` and Casts

Instead of:

```csharp
HourlyEmployee h = emp as HourlyEmployee;
if (h != null) { ... }
```

Use pattern matching:

```csharp
if (emp is HourlyEmployee hourly) { ... }
```

It’s cleaner and safer.

---

## 9.8 Chapter Summary

In this chapter, you’ve mastered the core OOP concepts of inheritance and polymorphism:

- **Inheritance** allows you to create derived classes that extend base classes, reusing code and establishing “is‑a” relationships.
- The **`base` keyword** lets you access base class members and call base constructors.
- **Virtual and override** methods enable derived classes to provide their own implementations while preserving polymorphic behavior.
- **Abstract classes and methods** define a contract that derived classes must fulfill, and prevent instantiation of the base class.
- **Sealed classes and methods** stop further inheritance or overriding.
- **Polymorphism** lets you treat objects of derived types as base type objects, with method calls resolved at runtime based on the actual object type.

These tools are fundamental to designing flexible, maintainable object‑oriented systems.

In the next chapter, **Interfaces – Defining Contracts**, you’ll learn about interfaces, which are similar to abstract classes but with multiple inheritance capabilities and a pure contract focus. Interfaces are widely used in C# to achieve loose coupling and enable powerful design patterns.

**Exercises:**

1. Create a base class `Animal` with a virtual method `MakeSound()`. Derive `Dog` and `Cat` classes that override it. Write a program that stores a list of animals and makes each sound.
2. Design an abstract class `Shape` with an abstract method `GetArea()`. Implement `Circle`, `Rectangle`, and `Triangle` derived classes. Test polymorphism by iterating through a list of shapes.
3. Add a `sealed` class `FinalShape` that derives from `Rectangle` and try to derive from it – see the compiler error.
4. In the employee example, add a new employee type `InternEmployee` that has a fixed stipend. Add it to the list and see how the existing code handles it without changes.
5. Experiment with calling base methods in overrides. Create a base method that logs something, override it in a derived class, and call `base.Method()`.

Now, get ready to dive into interfaces in Chapter 10!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='8. constructors_object_initialization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='10. interfaces_defining_contracts.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
